<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/github_pr_writer_haystack2_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This notebook demonstrates the versatility of Haystack 2.x framework in integrating with any OpenAPI specification service, exemplified here using automated GitHub Pull Request writing. It highlights how we can dynamically invoke any OpenAPI services and incorporate their outputs into the context of a Large Language Model (LLM), showcasing on-demand, service-based Retrieval-Augmented Generation (RAG).

## 1. Setup

This notebook demos GitHub Pull Request (PR) text generation.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [ ]:
!pip uninstall -y llmx

In [1]:
!pip install -q jsonref openapi3 haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [8]:
import getpass
import os
import requests

from haystack import Pipeline
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret

## 2. API Key Input and System Initialization

Begin by entering your OpenAI API key. Following this step, we initialize a system message for the GitHub PR Expert.

In [17]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")
github_token = getpass.getpass("Enter GitHub token:")

Enter LLM provider api key:··········
Enter GitHub token:··········


## 3. Pipeline Creation and Configuration

This section involves setting up the core components of the Haystack 2.x pipeline, which includes the OpenAPIServiceToFunctions, OpenAIChatGenerator, and OpenAPIServiceConnector. These components are connected to create a pipeline that processes and interprets the GitHub PR commands and data.

In [34]:
gen_func_pipeline = Pipeline()
gen_func_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())


system_message = requests.get("https://bit.ly/auto_pr_writer_system_prompt").text


functions_result = gen_func_pipeline.run(data={"sources":["https://bit.ly/github_compare"],
                                               "system_messages":[system_message]})

In [24]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(service_auths={"Github API": github_token}))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

gen_pipe = Pipeline()
gen_pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk))

## 4. User Input and PR Command Processing

Here, the user can input specific GitHub PR commands. Make sure to mention
project, repo and the branches involved.

In [11]:
user_prompt = input("Enter your GitHub PR command: ")
#Example: Compare branches main and test/benchmarks2.0, in project deepset-ai, repo haystack
#Example: Compare branches main and rafaelpadilla:add_bbox_transformations in project huggingface repo transformers

Enter your GitHub PR command: Compare branches main and test/benchmarks2.0, in project deepset-ai, repo haystack


In [20]:
messages = [ChatMessage.from_system("You are a helpful assistant capable of function calling."),
            ChatMessage.from_user(user_prompt)]

## 5. Processing OpenAPI Specification and GitHub Service Invocation
In this step, the notebook retrieves the OpenAPI specification for the GitHub compare branches service. This specification is then transformed into OpenAI function definitions. When a user inputs a command, the LLM generates service information parameters from this input. These parameters are used to dynamically invoke the GitHub compare branches service, allowing for real-time, context-sensitive interactions with GitHub's API.


But before we do that let's review the GitHub OpenAPI service definition.


In [21]:
import json
import requests
from IPython.display import HTML

def render(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

response = requests.get("https://bit.ly/github_compare")
response.raise_for_status()
render(response.json())

In [22]:
open_api_doc = functions_result["spec_to_functions"]["documents"][0]
openai_functions_definition = json.loads(open_api_doc.content)
openapi_spec = open_api_doc.meta["spec"]

In [25]:
# The fetched data, which includes details like PR commits, descriptions, and author information
tools_param = [{"type": "function", "function": openai_functions_definition}]
tool_choice = {"type": "function", "function": {"name": openai_functions_definition["name"]}}

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"tools": tools_param,
                                                                       "tool_choice": tool_choice},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generating Github PR Text with GPT-4 Model

Using the latest GPT-4 model (gpt-4-1106-preview), this section generates the textual content of the GitHub PR using the GitHub service data as context.

In [36]:

gen_pipe = Pipeline()
gen_pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk))

github_pr_prompt_messages = [ChatMessage.from_system(system_message)] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

### Why:
The change is motivated by the need to establish a continuous benchmarking system for the project. This includes the implementation of GitHub Actions for automatic benchmark running, collection of metrics related to benchmarking tasks, and integration with the Datadog platform for metrics reporting and visualization. Effective benchmarking is crucial for performance evaluation and ensures that any code changes do not degrade the quality or performance of the system.

### What:
- **GitHub Actions Workflow**: A new GitHub Actions workflow file added for running benchmarks and sending results to Datadog.
- **Datadog Metric Handlers**: Scripts added to handle custom metrics for Datadog, including various tagging and data collection utilities.
- **Benchmarking Scripts**: Python scripts for running indexing and retrieval operations, designed to generate performance metrics.
- **Benchmarking Pipelines**: Added YAML configuration files for indexing and retrieval operations that will b

##7. Displaying the Generated PR Text

Although we also streamed GitHub PR text, the generated GitHub PR text is displayed below in a special markdown component.

In [37]:
from IPython.display import Markdown
Markdown(final_result["llm"]["replies"][0].content)

### Why:
The change is motivated by the need to establish a continuous benchmarking system for the project. This includes the implementation of GitHub Actions for automatic benchmark running, collection of metrics related to benchmarking tasks, and integration with the Datadog platform for metrics reporting and visualization. Effective benchmarking is crucial for performance evaluation and ensures that any code changes do not degrade the quality or performance of the system.

### What:
- **GitHub Actions Workflow**: A new GitHub Actions workflow file added for running benchmarks and sending results to Datadog.
- **Datadog Metric Handlers**: Scripts added to handle custom metrics for Datadog, including various tagging and data collection utilities.
- **Benchmarking Scripts**: Python scripts for running indexing and retrieval operations, designed to generate performance metrics.
- **Benchmarking Pipelines**: Added YAML configuration files for indexing and retrieval operations that will be used in benchmarking tests.
- **Utility Functions**: Utility Python script added that facilitates downloading datasets and queries used for benchmarking purposes.

### How can it be used:
- The GitHub Actions workflow can automatically run benchmarking tasks on a schedule or upon request.
- The Datadog metric handlers can collect and send various performance metrics to Datadog where they can be tracked over time.
- Benchmarking scripts can be executed to measure the performance of different components or operations within the project.
- Pipelines defined in YAML files streamline benchmark setup, ensuring consistent environments for tests.
- Utility functions support efficient setup for benchmarking tasks by managing datasets.

### How did you test it:
The commits do not provide explicit details on test executions for the newly added automation and scripts. It would be advisable to enforce the following testing strategy:
- **Unit Testing**: Validate individual functions within utility scripts and metric handlers.
- **Integration Testing**: Ensure that the complete workflow, including GitHub Actions, benchmarking scripts, and metric handling, works as expected in an integrated manner.
- **Performance Testing**: Actual execution of the benchmarking scripts should result in consistent and accurate performance metrics.
- **End-to-End Testing**: Run the entire CI pipeline including the cleanup tasks to verify that it functions end-to-end.

### Notes for the reviewer:
- Special attention should be given to ensuring that GitHub Action secrets and environment variables are correctly configured.
- Validation is required to ensure that Datadog metrics are correctly formed and submitted.
- It might be beneficial to check for Python script compatibility with the project's existing codebase.
- Review the scheduling times and triggers for the GitHub Actions workflow to confirm they meet the project's benchmarking frequency requirements.
- Ensure that all newly added scripts adhere to the project's coding and documentation standards.

## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fgithub_pr_writer_haystack2_x.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://x.com/vladblagoje